# 마포구 (1092)
- 작성자 : 황다은
- 작성일자 : 2020년 9월 22일
- route data 중 exd, rgt, msk, ste 존재하나 전처리하는게 나의능력을 벗어남.. ㅠㅠ

In [2]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [3]:
df = pd.read_csv("200922one_shot.csv")
soup = BeautifulSoup(df['html'][0], 'html.parser')

### 2. 간단한 전처리 하기

In [204]:
rows = soup.select('div.tbl_wrap > table')

table = pd.read_html(str(rows[0]))[0]

' 8. 31(월)'

In [ ]:
rows = soup.select('div.tbl_wrap > table > tbody > tr')
rows[10].select('p')[1]

### 3. 저장할 DataFrame 미리 만들기

In [208]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt','age','sex']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [209]:
for i in tqdm(range(len(table))):
    subtable = table.iloc[i,:]
    if type(subtable.인적사항) != float:
        numid = subtable.인적사항.split(' ')[1].replace('번','')
        num = '1092' + '0' * (5- len(numid)) + numid
        cnd = subtable.확진일
        rsd = ''; adt = ''; age = ''; sex = ''
        if len(subtable.인적사항.split(' ')) > 2:
            dch = subtable.인적사항.split(' ')[3]
        else:
            dch = ''     
        if '삭제' in subtable.이동경로:
            syd = ''; syy = ''; atn =''; pth = '';
        else:
            if '○' in subtable.이동경로:
                try:
                    atnnum = len(subtable.이동경로.split('○'))
                    syd = subtable.이동경로.split('○')[0].split(':')[1].split('  *')[0] #syd
                    pth = subtable.이동경로.split('○')[0].split(':')[2] #pth
                    atn = subtable.이동경로.split('○')[atnnum-1].split('* ')[1]

                    if '무증상' in syd:
                        syd = ''; syy = 0
                    else:
                        syy = 1
                except:
                    syd = ''; syy = ''; pth = ''; atn = '';
            else:
                atn = ''
                syd = subtable.이동경로.split(':')[1].split('  *')[0] #syd
                pth = subtable.이동경로.split(':')[2]
                if '무증상' in syd:
                    syd = ''; syy = 0
                else:
                    syy = 1
        new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt, age, sex)], columns = pat_col)
        patient = pd.concat([patient, new_pat])
        order = 1; rgl = ''; rgt = ''; frm = ''; exd = ''; msk = ''; ste = ''; mob ='';
        new_route =pd.DataFrame([(num, order, dat, rgl, rgt, frm, exd, msk, ste, mob)], columns = rou_col)
        route = pd.concat([route, new_route])

100%|████████████████████████████████████████████████████████████████████████████████| 171/171 [00:03<00:00, 52.19it/s]


In [210]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt,age,sex
0,109200167,2020-09-22,타지역 확진자 접촉,1,9. 20(일),20. 9. 22.,,,,,,
0,109200166,2020-09-22,확인 중,1,9. 18(금),20. 9. 21.,,,,,,
0,109200165,2020-09-22,확인 중,1,9. 18(금),20. 9. 21.,,,,,,
0,109200164,2020-09-22,마포구 162번 확진자 접촉,0,,20. 9. 21.,,9월 21일 16시 생활치료센터 입소,,,,
0,109200163,2020-09-22,마포구 150번 확진자 접촉,1,9. 15(화),20. 9. 21.,,9월 21일 15시25분 생활치료센터 입소,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
0,109200005,2020-09-22,,,,20. 3. 10.,,,완치,,,
0,109200004,2020-09-22,,,,20. 3. 10.,,,완치,,,
0,109200003,2020-09-22,,,,20. 3. 10.,,,완치,,,
0,109200002,2020-09-22,,,,20. 3. 10.,,,완치,,,


In [211]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,109200167,1,2020-09-22,,,,,,,
0,109200166,1,2020-09-22,,,,,,,
0,109200165,1,2020-09-22,,,,,,,
0,109200164,1,2020-09-22,,,,,,,
0,109200163,1,2020-09-22,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
0,109200005,1,2020-09-22,,,,,,,
0,109200004,1,2020-09-22,,,,,,,
0,109200003,1,2020-09-22,,,,,,,
0,109200002,1,2020-09-22,,,,,,,
